In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18224659
paper_name = 'shima_takagi_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Air-drying stress.xlsx', sheet_name='air-drying', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4715 x 5


In [7]:
original_data.head()

,ORF,GENE,Growth ratea,Unnamed: 3,DESCRIPTION (MIPS)
0,NaN,NaN,BM/BW,(DM/DW)/(BM/BW),NaN
1,YAL002W,VPS8,0.974735,1.05672,"vacuolar sorting protein, 134 kD"
2,YAL004W,NaN,0.987146,0.574597,questionable protein
3,YAL005C,SSA1,1.01242,1.02328,"heat shock protein of HSP70 family, cytosolic"
4,YAL007C,ERP2,1.01124,1.03665,p24 protein involved in membrane trafficking


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                           ORF GENE  \
index_input                                                           
0                                                          NaN  NaN   
4714         aSee the reference for detail. ND, Not determi...  NaN   

            Growth ratea       Unnamed: 3 DESCRIPTION (MIPS)  \
index_input                                                    
0                  BM/BW  (DM/DW)/(BM/BW)                NaN   
4714                 NaN              NaN                NaN   

                                                  orf  
index_input                                            
0                                                 NAN  
4714         ASEETHEREFERENCEFORDETAILNDNOTDETERMINED  


In [12]:
original_data = original_data.loc[t,:]

In [14]:
original_data['data'] = pd.to_numeric(original_data['Unnamed: 3'], errors='coerce')

In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(4688, 1)

# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [504]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,504
data_type,value
orf,
YAL002W,1.056721
YAL004W,0.574597
YAL005C,1.023282
YAL007C,1.036655
YAL008W,0.493508


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,504
,data_type,value
gene_id,orf,
2,YAL002W,1.056721
1863,YAL004W,0.574597
4,YAL005C,1.023282
5,YAL007C,1.036655
6,YAL008W,0.493508


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            504          
data_type           value    valuez
gene_id orf                        
2       YAL002W  1.056721  0.338969
1863    YAL004W  0.574597 -1.831706
4       YAL005C  1.023282  0.188419
5       YAL007C  1.036655  0.248626
6       YAL008W  0.493508 -2.196798

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18224659...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.86s/it]

Updating the data_modified_on field...
